In [ ]:
import os
from google.colab import userdata

# Set your OpenAI API key
os.environ["OPENAI_API_KEY"] = userdata.get('openai-key')

In [ ]:
!pip install scipy
!pip install text-to-number
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.9/312.9 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.9 MB/s eta 0:00:00


In [ ]:
import os
from openai import OpenAI

client = OpenAI()

# Initialize the OpenAI client with your API key
OpenAI.api_key = os.environ["OPENAI_API_KEY"]

def parse_output_to_dict(output):
    # Initialize the dictionary
    data_dict = {}

    # Split the output into lines
    lines = output.split('\n')

    # Extract and parse each line
    for line in lines:
        if line.startswith("Category:"):
            data_dict['Category'] = line.split(': ')[1]
        elif line.startswith("Number of categories:"):
            data_dict['Number of categories'] = int(line.split(': ')[1])
        elif line.startswith("Categories:"):
            data_dict['Categories'] = line.split(': ')[1].split(', ')
        elif line.startswith("Type:"):
            data_dict['Type'] = line.split(': ')[1]
        elif line.startswith("Number of types:"):
            data_dict['Number of types'] = int(line.split(': ')[1])
        elif line.startswith("Types:"):
            data_dict['Types'] = line.split(': ')[1].split(', ')
        elif line.startswith("Observed frequency:"):
          # Process observed frequencies to convert them into a list of lists of integers
          frequencies = line.split(': ')[1].strip('[]').split('], [')
          data_dict['Observed frequency'] = [list(map(int, freq.strip('[]').split(', '))) for freq in frequencies]

    return data_dict

def get_output(question):
    # prompt = f"Given the question: '{question}', please categorize the question and provide the observed frequencies. For example, if the question is about a male buying a car, the category could be 'Male' and the type 'bought a car'. The observed frequencies could be '207, 811, 65, 984'."
    prompt = f"Given the question: '{question}', please categorize the question and provide the observed frequencies. For example, if the question is about a male buying a car, the category could be 'Male' and the type 'bought a car'. The observed frequencies could be '207, 811, 65, 984'.\n\nCategory: Gender\nNumber of categories: 2\nCategories: Male, Female\nType: Purchase Decision\nNumber of types: 2\nTypes: Bought, Not Bought\nObserved frequency: [[207, 811], [65, 984]]"

    # Make an API call to OpenAI using the appropriate method
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",  # Adjust model as necessary
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=150,
        temperature=0.5
    )

    # Extracting and processing the response according to your structure
    # output = response.choices[0].message['content'].strip()  # Adjusted access to 'content'
    output = response.choices[0].message.content.strip()  # Adjusted access to 'content'
    # print(f"output: {output}")
    output=parse_output_to_dict(output)
    # response_message = response.choices[0].message.content

    return output

# Example usage
question = "A researcher interviewed 2067 people and asked whether they were the primary decision makers in the household when buying a new car last year. Two hundred seven were men and had bought a new car last year. Sixty-five were women and had bought a new car last year. Eight hundred eleven of the responses were from men who did not buy a car last year. Nine hundred eighty-four were from women who did not buy a car last year. Use these data to determine whether gender is independent of being a major decision maker in purchasing a car last year. Let a = .05."
output = get_output(question)
print(output['Observed frequency'][0][0])
isinstance(output['Observed frequency'][0][0],str)

207


False

In [ ]:
import spacy
from spacy.matcher import Matcher
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
import pandas as pd
import re
import scipy
from scipy.stats import chi2
import numpy as np
from text_to_number import text_to_number as t2n

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

nlp = spacy.load("en_core_web_sm")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
text="A researcher interviewed 2067 people and asked whether they were the primary decision makers in the household when buying a new car last year. Two hundred seven were men and had bought a new car last year. Sixty-five were women and had bought a new car last year. Eight hundred eleven of the responses were from men who did not buy a car last year. Nine hundred eighty-four were from women who did not buy a car last year. Use these data to determine whether gender is independent of being a major decision maker in purchasing a car last year. Let a = .05."

In [ ]:
print(text)
print(t2n(text))

A researcher interviewed 2067 people and asked whether they were the primary decision makers in the household when buying a new car last year. Two hundred seven were men and had bought a new car last year. Sixty-five were women and had bought a new car last year. Eight hundred eleven of the responses were from men who did not buy a car last year. Nine hundred eighty-four were from women who did not buy a car last year. Use these data to determine whether gender is independent of being a major decision maker in purchasing a car last year. Let a = .05.
A researcher interviewed 2067 people and asked whether they were the primary decision makers in the household when buying a new car last year. 207 were men and had bought a new car last year. 65 were women and had bought a new car last year. 811 of the responses were from men who did not buy a car last year. 984 were from women who did not buy a car last year. Use these data to determine whether gender is independent of being a major decis

In [ ]:
def isChi(text,fo,fe):
  doc = nlp(text)
  chi_squared_matcher = Matcher(nlp.vocab)
  chi_squared_patterns = [
    [{"LOWER": "chi"}, {"LOWER": "squared"}],
    [{"LOWER": "association"}],
    [{"LOWER": "chi-square"}],
    [{"LOWER": "chisquare"}],
    [{"LOWER": "observed"}],
    [{"LOWER": "expected"}],
    [{"LOWER": "poisson"}],
    [{"LOWER": "fits"}],
    [{"LOWER": "fit"}],
    [{"LOWER": "independent"}],
    [{"LOWER": "chi2"}],
    [{"LOWER": "χ2"}],
    [{"LOWER": "pearson's"}, {"LOWER": "chi-squared"}],
    [{"LOWER": "pearson"}, {"LOWER": "chi-square"}],
    [{"LOWER": "contingency"}],
    [{"LOWER": "goodness-of-fit"}],
    [{"LOWER": "independence"}],
    [{"LOWER": "categorical"}],
    [{"LOWER": "χ-square"}],
    [{"LOWER": "chi"}],
    [{"LOWER": "chi2-square"}],
    [{"LOWER": "chi-sq"}],
    [{"LOWER": "χ2-square"}],
    [{"LOWER": "χ-square"}, {"LOWER": "analysis"}],
    [{"LOWER": "pearson's"}],
    [{"LOWER": "pearson"}],
    [{"LOWER": "contingency"}, {"LOWER": "analysis"}],
    [{"LOWER": "goodness"}, {"LOWER": "of"}, {"LOWER": "fit"}, {"LOWER": "analysis"}],
    [{"LOWER": "independence"}, {"LOWER": "analysis"}],\
    [{"LOWER": "association"}, {"LOWER": "analysis"}],
    [{"LOWER": "χ2"}],
    [{"LOWER": "chi-squared"}, {"LOWER": "homogeneity"}],
    [{"LOWER": "categorical"}, {"LOWER": "variable"}, {"LOWER": "analysis"}],
    [{"LOWER": "chi-square"}, {"LOWER": "p-value"}],
    [{"LOWER": "chi-squared"}, {"LOWER": "significance"}],
    [{"LOWER": "chi-square"}, {"LOWER": "critical"}, {"LOWER": "region"}],
    [{"LOWER": "categorical"}, {"LOWER": "relationship"}],
    [{"LOWER": "chi-sq"}],
    [{"LOWER": "χ2"}, {"LOWER": "goodness-of-fit"}],
    [{"LOWER": "χ-square"}, {"LOWER": "of"}, {"LOWER": "independence"}],
    [{"LOWER": "chi-square"}, {"LOWER": "analysis"}],
    [{"LOWER": "chi2"}],
    [{"LOWER": "χ2"}],
    [{"LOWER": "χ2"}, {"LOWER": "contingency"}],
    [{"LOWER": "discrete"}],
    [{"LOWER": "chi-squared"}],
    [{"LOWER": "chi-square"}, {"LOWER": "statistic"}],
    [{"LOWER": "chi-square"}, {"LOWER": "p-value"}],
    [{"LOWER": "chi-square"}, {"LOWER": "critical"}, {"LOWER": "value"}],
    [{"LOWER": "χ-square"}, {"LOWER": "distribution"}],
    [{"LOWER": "chi-sq"}],
    [{"LOWER": "χ2"}, {"LOWER": "goodness-of-fit"}],
    [{"LOWER": "χ-square"}, {"LOWER": "of"}, {"LOWER": "independence"}],
    [{"LOWER": "chi-square"}, {"LOWER": "analysis"}],
    [{"LOWER": "chi2"}],
    [{"LOWER": "χ2"}],
    [{"LOWER": "χ2"}, {"LOWER": "contingency"}],
    [{"LOWER": "discrete"}],
    [{"LOWER": "chi-squared"}],
    [{"LOWER": "chi-square"}, {"LOWER": "statistic"}],
    [{"LOWER": "chi-square"}, {"LOWER": "p-value"}],
    [{"LOWER": "chi-square"}, {"LOWER": "critical"}, {"LOWER": "value"}],
    [{"LOWER": "χ-square"}, {"LOWER": "distribution"}],
    [{"LOWER": "χ2"}],
    [{"LOWER": "chi-squared"}, {"LOWER": "for"}, {"LOWER": "independence"}],
    [{"LOWER": "chi-squared"}, {"LOWER": "contingency"}],
    [{"LOWER": "chi-square"}, {"LOWER": "distribution"}],
    [{"LOWER": "categorical"}, {"LOWER": "analysis"}],
    [{"LOWER": "crosstab"}],
    [{"LOWER": "chi-square"}, {"LOWER": "goodness-of-fit"}],
    [{"LOWER": "chi-squared"}, {"LOWER": "homogeneity"}],
    [{"LOWER": "categorical"}, {"LOWER": "variable"}, {"LOWER": "analysis"}],
    [{"LOWER": "chi-square"}, {"LOWER": "p-value"}],
    [{"LOWER": "chi-squared"}, {"LOWER": "significance"}],
    [{"LOWER": "chi-square"}, {"LOWER": "critical"}, {"LOWER": "region"}],
    [{"LOWER": "categorical"}, {"LOWER": "relationship"}],
    [{"LOWER": "chi-square"}, {"LOWER": "significance"}],
    [{"LOWER": "chi-square"}, {"LOWER": "critical"}, {"LOWER": "values"}],
    [{"LOWER": "chi-square"}, {"LOWER": "degrees"}, {"LOWER": "freedom"}],
    [{"LOWER": "chi-square"}, {"LOWER": "independence"}],
    [{"LOWER": "contingency"}],
    [{"LOWER": "categorical"}, {"LOWER": "association"}],
    [{"LOWER": "chi-square"}, {"LOWER": "null"}, {"LOWER": "hypothesis"}],
    [{"LOWER": "chi-square"}, {"LOWER": "alternative"}, {"LOWER": "hypothesis"}],
    [{"LOWER": "chi-square"}, {"LOWER": "hypothesis"}],
    [{"LOWER": "chi-squared"}, {"LOWER": "for"}, {"LOWER": "goodness-of-fit"}],
    [{"LOWER": "chi-square"}, {"LOWER": "contingency"}],
    [{"LOWER": "categorical"}],
    [{"LOWER": "chi-square"}, {"LOWER": "statistical"}],
    [{"LOWER": "pearson"}, {"LOWER": "chi-squared"}, {"LOWER": "statistic"}],
    [{"LOWER": "contingency"}, {"LOWER": "independence"}],
    [{"LOWER": "χ2-square"}],
    [{"LOWER": "association"}, {"LOWER": "chi-square"}],
    [{"LOWER": "categorical"}, {"LOWER": "independence"}],
    [{"LOWER": "χ2"}, {"LOWER": "for"}, {"LOWER": "homogeneity"}],
    [{"LOWER": "chi-square"}, {"LOWER": "analysis"}],
    [{"LOWER": "chi-squared"}, {"LOWER": "for"}, {"LOWER": "categorical"}, {"LOWER": "variables"}],
    [{"LOWER": "χ-square"}, {"LOWER": "for"}, {"LOWER": "non-parametric"}],
    [{"LOWER": "chi-square"}, {"LOWER": "for"}, {"LOWER": "observed"}, {"LOWER": "and"}, {"LOWER": "expected"}],
  ]

  t_test_pattern = [{"LOWER": "t-test"}]
  anova_pattern = [{"LOWER": "anova"}]
  regression_pattern = [{"LOWER": "regression"}]
  correlation_pattern = [{"LOWER": "correlation"}]
  z_test_pattern = [{"LOWER": "z-test"}]
  f_test_pattern = [{"LOWER": "f-test"}, {"LOWER": "analysis"}, {"LOWER": "variance"}]
  wilcoxon_pattern = [{"LOWER": "wilcoxon"}, {"LOWER": "rank-sum"}]
  mann_whitney_pattern = [{"LOWER": "mann-whitney"}, {"LOWER": "u"}]
  kruskal_wallis_pattern = [{"LOWER": "kruskal-wallis"}]
  mcnemar_pattern = [{"LOWER": "mcnemar"}]
  logistic_regression_pattern = [{"LOWER": "logistic"}, {"LOWER": "regression"}]
  cox_proportional_hazards_pattern = [{"LOWER": "cox"}, {"LOWER": "proportional-hazards"}]
  mantel_haenszel_pattern = [{"LOWER": "mantel-haenszel"}]
  kolmogorov_smirnov_pattern = [{"LOWER": "kolmogorov-smirnov"}]
  chi_squared_matcher.add("ChiSquaredTest", chi_squared_patterns)

  chi_squared_matches = chi_squared_matcher(doc)

  # Rule-based matching for other tests
  other_tests_matcher = Matcher(nlp.vocab)
  other_tests_matcher.add("OtherTests", [
      t_test_pattern, anova_pattern, regression_pattern, correlation_pattern,
      z_test_pattern, f_test_pattern, wilcoxon_pattern, mann_whitney_pattern,
      kruskal_wallis_pattern, mcnemar_pattern,
      logistic_regression_pattern, cox_proportional_hazards_pattern,
      mantel_haenszel_pattern, kolmogorov_smirnov_pattern
  ])
  other_tests_matches = other_tests_matcher(doc)
  # Check if there are no matches for other tests and there are chi-squared test matches
  return len(other_tests_matches) == 0 and len(chi_squared_matches) > 0

def isGOF(text, fo, fe):
    goodness_of_fit_patterns = [
        r'chi[\s-]*square[\s-]*goodness[\s-]*of[\s-]*fit[\s-]*test',
        r'observed[\s-]*frequencies',
        r'expected[\s-]*frequencies',
        r'fit[\s-]*test',
        r'gof',
        r'goodness[\s-]*fit',
        r'frequency[\s-]*distribution[\s-]*comparison',
        r'distribution[\s-]*fit',
        r'fit[\s-]*observed[\s-]*expected',
        r'chi[\s-]*squared[\s-]*fit',
        r'chi[\s-]*square[\s-]*distribution[\s-]*test',
        r'frequency[\s-]*fit',
        r'fit[\s-]*categorical[\s-]*data',
        r'categorical[\s-]*distribution[\s-]*test',
        r'fit[\s-]*test[\s-]*for[\s-]*distributions',
        r'chi[\s-]*2[\s-]*goodness',
        r'χ[\s-]*squared[\s-]*goodness',
        r'chi[\s-]*square[\s-]*g[\s-]*o[\s-]*f',
        r'chi[\s-]*test[\s-]*for[\s-]*fit',
        r'fitting[\s-]*observed[\s-]*to[\s-]*expected',
        r'conformity[\s-]*of[\s-]*observed[\s-]*with[\s-]*expected',
        r'how[\s-]*well[\s-]*data[\s-]*fits',
        r'comparing[\s-]*observed[\s-]*to[\s-]*expected',
        r'uniform[\s-]*distribution',
        r'uniformly[\s-]*distributed',
        r'poisson[\s-]*distributed',
        r'normally[\s-]*distributed',
        r'normal[\s-]*distribution'
    ]

    # Check for goodness-of-fit patterns
    for pattern in goodness_of_fit_patterns:
        if re.search(pattern, text, re.IGNORECASE):
            return True

    # If no patterns match, it's not a GOF type question
    return False

def isTFI(text,fo,fe):
  independence_patterns = [
    r'chi[\s-]*square[\s-]*test[\s-]*of[\s-]*independence',
    r'contingency[\s-]*table',
    r'independence[\s-]*test',
    r'association[\s-]*test',
    r'relationship[\s-]*between[\s-]*categorical',
    r'categorical[\s-]*variables[\s-]*association',
    r'two[\s-]*way[\s-]*table',
    r'two[\s-]*way[\s-]*anova',
    r'cross[\s-]*tabulation',
    r'χ[\s-]*square[\s-]*for[\s-]*independence',
    r'χ[\s-]*2[\s-]*independence',
    r'chi[\s-]*2[\s-]*association',
    r'χ[\s-]*squared[\s-]*test[\s-]*for[\s-]*association',
    r'chi[\s-]*square[\s-]*contingency',
    r'categorical[\s-]*data[\s-]*association',
    r'association[\s-]*between[\s-]*two[\s-]*variables',
    r'frequency[\s-]*data[\s-]*analysis',
    r'χ[\s-]*2[\s-]*test[\s-]*of[\s-]*association',
    # Added patterns to capture the use of "independent"
    r'independent[\s-]*of',
    r'whether[\s-]*.*[\s-]*is[\s-]*independent',
    r'independent[\s-]*variable',
    r'dependent[\s-]*and[\s-]*independent',
    r'variable[\s-]*is[\s-]*independent'
  ]

    # Check for test of independence patterns
  for pattern in independence_patterns:
    if re.search(pattern, text, re.IGNORECASE):
      return True

    # If no patterns match, it's not a TFI type question
  return False

def dataTableHasFe(fe):
  return len(fe)>0

def dataTableHasFo(fo):
  if(fo==[]):
    return False
  if(fo==''):
    return False
  return True

def textHasFe(text):
  return False

In [ ]:
import re

def extractAlpha(text):
    # Regular expressions to match different representations of alpha
  regex_patterns = [
    r"α\s*=\s*(0?\.\d+|\d+\.\d*)",  # Explicit mention with symbol, allowing for ".05" or "0.05"
    r"alpha\s*=\s*(0?\.\d+|\d+\.\d*)",  # Explicit mention with word
    r"at\s+a\s+significance\s+level\s+of\s+(0?\.\d+|\d+\.\d*)",  # Descriptive representation
    r"set\s+to\s+(0?\.\d+|\d+\.\d*)",  # Descriptive representation with "set to"
    r"chosen\s+a\s+significance\s+level\s+of\s+(0?\.\d+|\d+\.\d*)",  # Descriptive representation with "chosen a significance level of"
    r"with\s+α\s*=\s*(0?\.\d+|\d+\.\d*)",  # Symbolic representation
    r"at\s+a\s+level\s+of\s+(0?\.\d+|\d+\.\d*)",  # Alternative descriptive representation
    r"p\s*=\s*(0?\.\d+|\d+\.\d*)",  # Representation using 'p'
    r"probability\s*threshold\s*was\s*set\s*to\s*(0?\.\d+|\d+\.\d*)",  # More flexible representation using 'probability'
    r"a\s*=\s*(0?\.\d+|\d+\.\d*)",  # Explicit mention of 'a' with symbol
    r"s+\s*a\s*=\s*(0?\.\d+|\d+\.\d*)",  # Corrected typo for 's+a' to allow spaces between 's' and 'a'
    r"s+\s*alpha\s*=\s*(0?\.\d+|\d+\.\d*)"  # Corrected typo for 's+alpha' to allow spaces between 's' and 'alpha'
  ]

    # Search for alpha in the text using regular expressions
  for pattern in regex_patterns:
    match = re.search(pattern, text, re.IGNORECASE)
    if match:
      return float(match.group(1).replace(',', '.'))  # Replace comma with dot for consistency

    # If alpha is not found, return default value 0.05
  return 0.05

In [ ]:

def extractFo(text,fo):
  if dataTableHasFo(fo):
    return fo

  # extract from fo
  return extractFoFromText(text)

def extractFoFromText(text):
  # Pattern to find numeric values (both whole numbers and decimals)
  number_pattern = r'\b\d+(?:\.\d+)?\b'  # Use non-capturing group for decimal part

  # Assuming extractAlpha has already been called and its value stored
  alpha_value = extractAlpha(text)  # Call illustrative; in practice, avoid redundant calls

  # Find all numbers in the text
  all_numbers_matches = re.findall(number_pattern, text)

  # Safely convert non-empty matches to floats
  all_numbers_float = [float(match) for match in all_numbers_matches if match]

  # Filter out the alpha value if it appears as a number in the text
  observed_frequencies = [num for num in all_numbers_float if num != alpha_value]

  return observed_frequencies

def extractFe(text,fo,fe):
  print(fe)
  if (fe=='') or (fe==[]):
    return extractFeFromText(text,fo)

  if dataTableHasFe(fe):
    # extract From DataTable
    return fe

  return extractFeFromText(text,fo)

def extractFeFromText(text,fo):
  # switch case for different
  if(textHasFe(text)): #false for now
    # nlp
    return 110

  switch = identifyDistribution(text)
  fe=getFeFromFo(switch,fo)
  return fe

def getFeFromFo(distribution, fo):
    # Convert non-integer elements to integers or handle them appropriately
    fo = [int(item) if isinstance(item, (int, float)) else item[1] for item in fo]

    if distribution == 'uniform':
        total = sum(fo)
        n_categories = len(fo)
        return [total / n_categories for _ in fo]
    elif distribution == 'poisson':
        res = poisson(fo)
        return res
    elif distribution == 'binomial':
        # Placeholder: Implement based on n (trials) and p (success probability)
        return []
    elif distribution == 'normal':
        # Placeholder: Implement based on μ (mean) and σ (std dev)
        res=normal(fo)
        return res
    else:
        #just execute uniform
        total = sum(fo)
        n_categories = len(fo)
        return [total / n_categories for _ in fo]

def identifyDistribution(text):
    if 'uniform' in text.lower() or 'uniformly' in text.lower():
        print('uniform')
        return 'uniform'
    elif 'poisson' in text.lower():
        print('poisson')
        return 'poisson'
    elif 'binomial' in text.lower():
        print('binomial')
        return 'binomial'
    elif 'normal' in text.lower():
        print('normal')
        return 'normal'
    else:
        print('uniform because no other')
        return None

from math import exp, factorial

from scipy.stats import norm
import numpy as np

def normal(fo):
    # Estimate mean (μ) and standard deviation (σ) from fo
    total_observations = sum(freq for _, freq in fo)
    mean = sum(midpoint * freq for midpoint, freq in fo) / total_observations
    variance = sum(freq * ((midpoint - mean) ** 2) for midpoint, freq in fo) / total_observations
    std_dev = np.sqrt(variance)

    # Calculate expected frequencies
    fe = []
    for i, (midpoint, _) in enumerate(fo):
        if i == 0:
            # For the first category, assume lower bound is far enough to approximate -infinity
            lower_bound = -np.inf
        else:
            # Use the midpoint between this category and the previous one
            lower_bound = (fo[i][0] + fo[i-1][0]) / 2

        if i == len(fo) - 1:
            # For the last category, assume upper bound is far enough to approximate infinity
            upper_bound = np.inf
        else:
            # Use the midpoint between this category and the next one
            upper_bound = (fo[i][0] + fo[i+1][0]) / 2

        # Calculate cumulative probability for lower and upper bounds
        lower_cdf = norm.cdf(lower_bound, mean, std_dev)
        upper_cdf = norm.cdf(upper_bound, mean, std_dev)

        # Expected frequency for this category
        expected_frequency = (upper_cdf - lower_cdf) * total_observations
        fe.append(expected_frequency)

    return fe

# Example usage
# Suppose fo is given as [(midpoint, frequency), ...]
fo = [(15, 6), (25, 14), (35, 29), (45, 38), (55, 25), (65, 10), (75, 7)]
fe_normal = normal(fo)
print("Expected frequencies (Normal):", fe_normal)

def preprocess_fo(fo):
    # Check if the input is already in the expected format (list of tuples)
    if all(isinstance(item, tuple) for item in fo):
        return fo

    # If not, assume it's a list of frequencies starting from 0 arrivals
    return [(i, freq) for i, freq in enumerate(fo)]

def poisson(fo):
    # Preprocess input to ensure it's in the correct format
    fo = preprocess_fo(fo)

    # Calculate total number of observations
    N = sum(frequency for _, frequency in fo)

    # Calculate λ (lambda): average rate of occurrences
    lambda_ = sum(x * frequency for x, frequency in fo) / N
    print("λ =",lambda_)

    # Calculate expected probabilities and frequencies for each x using the Poisson formula
    fe = [(x, ((lambda_ ** x) * exp(-lambda_) / factorial(x)) * N) for x, _ in fo]

    # Return only the expected frequencies, discarding x
    expected_frequencies = [expected for _, expected in fe]

    return expected_frequencies

# # Example usage:
# fo = [207, 65, 811, 984]  # Assuming this represents frequencies of 0, 1, 2, 3... arrivals
# expected_frequencies = poisson(fo)
# print("Expected frequencies:", expected_frequencies)

# # Example usage
# # fo should be a list of tuples (x, observed frequency of x)
# # For example: [(0, 18), (1, 36), (2, 25), (3, 12), (4, 9)]
# fo = [(0, 18), (1, 36), (2, 25), (3, 12), (4, 9)]
# expected_frequencies = poisson(fo)
# print(expected_frequencies)

def extractN(text,dataTable):
  if dataTable=='':
    return extractNFromText(text)

  # get from data table counting fo
  return 10

def extractNFromText(text):
  # Nlp
  return 10

# Function to solve Chi-square goodness of fit test
import numpy as np
from scipy.stats import chi2

def solveGOF(fo, fe, n, alpha):
    # Define null and alternate hypothesis
    null_hypothesis = "Null Hypothesis (H0): Observed frequencies are consistent with the expected frequencies."
    alternate_hypothesis = "Alternate Hypothesis (H1): Observed frequencies differ significantly from expected frequencies."

    # Convert lists to numpy arrays for element-wise operations
    fo = np.array(fo)
    fe = np.array(fe)

    # Calculate Chi-square test statistic
    chi_square = np.sum((fo - fe)**2 / fe)

    # Determine degrees of freedom
    df = n - 1  # Adjusted based on the input parameter 'n', assuming it represents the number of categories

    # Calculate p-value
    p_value = 1 - chi2.cdf(chi_square, df)

    # Determine critical value
    critical_value = chi2.ppf(1 - alpha, df)

    # Output
    print("1) ")
    print(null_hypothesis)
    print(alternate_hypothesis)
    print("2) Chi Square test is appropriate")
    print("3) Alpha Value:", alpha)
    print("4) Critical Value:", critical_value)
    print("5) Chi-square value:", chi_square)
    print("6) Final Output:")

    # Compare p-value with significance level
    if p_value < alpha:
        print("Reject null hypothesis. Observed frequencies differ significantly from expected frequencies.")
    else:
        print("Fail to reject null hypothesis. No significant difference between observed and expected frequencies.")

# # Example data
# fo = np.array([60, 90, 75])  # Observed frequencies
# fe = np.array([70, 80, 75])  # Expected frequencies
# n = np.sum(fo)  # Total number of observations
# alpha = 0.05  # Significance level

# # Solve Chi-square goodness of fit test
# solveGOF(fo, fe, n, alpha)


Expected frequencies (Normal): [5.856349356809644, 14.757003637426454, 28.70552877332967, 35.06434698118453, 26.901667542358773, 12.959840939034082, 4.755262769856849]


In [ ]:

# New Functions for Test for Independence (TFI)
def extractFoTFI(text,fo):
    # Placeholder for TFI-specific observed frequency extraction
    if fo==[]:
      fo=get_output(text)['Observed frequency']
    return fo

def extractFeTFI(text, fo, fe):
    # Total number of observations
    n = sum(sum(row) for row in fo)

    # Calculate row totals
    row_totals = [sum(row) for row in fo]

    # Calculate column totals
    col_totals = [sum(fo[row][col] for row in range(len(fo))) for col in range(len(fo[0]))]

    # Calculate expected frequencies for each cell
    fe = [[(row_total * col_total) / n for col_total in col_totals] for row_total in row_totals]

    return fe

def extractNTFI(text,fo,fe):
    # Placeholder for TFI-specific total count extraction
    return 20

def solveTFI(fo, fe, df, alpha):
    print("Starting Test for Independence (TFI) analysis...")

    # Convert lists to numpy arrays for vectorized operations
    fo = np.array(fo)
    fe = np.array(fe)

    # Step 1: Calculate the Chi-Square statistic
    chi2_stat = np.sum((fo - fe) ** 2 / fe)
    print(f"\nStep 1: Chi-Square Statistic Calculation")
    print(f"Calculated Chi-Square Statistic: {chi2_stat:.4f}")

    # Step 2: Calculate the p-value
    p_value = 1 - scipy.stats.chi2.cdf(chi2_stat, df)
    print(f"\nStep 2: P-value Calculation")
    print(f"Calculated P-value: {p_value:.4g}")  # Using scientific notation if needed

    # Step 3: Calculate the critical value
    critical_value = scipy.stats.chi2.ppf(1 - alpha, df)
    print(f"\nStep 3: Critical Value Calculation")
    print(f"Calculated Critical Value: {critical_value:.4f}")

    # Step 4: Determine significance based on chi-square statistic and critical value
    is_significant_chi2 = chi2_stat > critical_value
    result_msg_chi2 = "significant" if is_significant_chi2 else "not significant"
    print(f"\nStep 4: Significance Testing Based on Chi-Square Statistic")
    print(f"Using alpha level of {alpha}, the chi-square statistic result is determined to be {result_msg_chi2}.")

    # Step 5: Determine significance based on p-value
    is_significant_p = p_value < alpha
    result_msg_p = "significant" if is_significant_p else "not significant"
    print(f"\nStep 5: Significance Testing Based on P-value")
    print(f"Using alpha level of {alpha}, the p-value result is determined to be {result_msg_p}.")

    # Final result
    print(f"\nFinal Result: The association between the variables is {result_msg_p} based on p-value and {result_msg_chi2} based on chi-square statistic.")
    print(f"Chi-Square Statistic: {chi2_stat:.4f}, P-value: {p_value:.4g}, Critical Value: {critical_value:.4f}, Significance Level (Alpha): {alpha}")

    return chi2_stat, p_value, critical_value, is_significant_p, is_significant_chi2


def extractDfTfi(fo):
    # Validate fo to ensure it's a list of lists
    if not isinstance(fo, list) or not all(isinstance(row, list) for row in fo):
        raise ValueError("fo must be a two-dimensional list.")

    r = len(fo)
    c = len(fo[0]) if r > 0 else 0
    df = (r - 1) * (c - 1)

    return df

In [ ]:
def preprocessfo(fo):
    # Check if 'fo' is a list of tuples
    if all(isinstance(item, tuple) and len(item) == 2 for item in fo):
        # Extract and return the second element from each tuple
        return [frequency for _, frequency in fo]
    else:
        # If 'fo' is not a list of tuples, return it unchanged
        return fo

def preprocessfe(fe, fo):
    if not isinstance(fe, list):
        fe = [fe]

    sum_fo = sum(fo)
    sum_fe = sum(fe)
    if 99 <= sum_fe <= 101:
        multiplier = sum_fo / sum_fe
        new_fe = [round(frequency * multiplier) for frequency in fe]
        return new_fe
    else:
        return fe

# # Example usage
# fo_tuples = [(1, 10), (2, 20), (3, 30)]
# fo_single = [10, 20, 30]

# # Should return [10, 20, 30]
# print(preprocessfo(fo_tuples))

# # Should also return [10, 20, 30] (unchanged)
# print(preprocessfo(fo_single))

def main(text, fo,fe):
    text=t2n(text)
    if not isChi(text, fo,fe):
        print("Not chi-square question")
        return

    alpha = extractAlpha(text)  # Common to both tests

    if isGOF(text,fo,fe):
        fo = extractFo(text,fo)
        fe = extractFe(text,fo,fe)
        fo = preprocessfo(fo)
        fe = preprocessfe(fe, fo)
        n = extractN(text,fo)
        print("fo",fo)
        print("fe",fe)
        print("n",n)
        result_gof = solveGOF(fo, fe, n, alpha)

    if isTFI(text,fo,fe):
        fo_tfi = extractFoTFI(text,fo)
        fe_tfi = extractFeTFI(text,fo_tfi,fe)
        n_tfi = extractNTFI(text,fo_tfi,fe)
        df_tfi = extractDfTfi(fo_tfi)
        print("fo",fo_tfi)
        print("fe",fe_tfi)
        print("n",n_tfi)
        print("df",df_tfi)
        result_tfi = solveTFI(fo_tfi, fe_tfi, df_tfi, alpha)

        print(f"Test for Independence Result: {result_tfi}")


In [ ]:
if __name__=='__main__':
  text='''
Use the following contingency table and the chi-square test of independence to determine whether social class is independent of number of children in a family. Let a = .05.
  '''
  fo=[[7,18,6,9],[38,23,34,97],[58,47,31,30]]
  fe=[]
  typ=""
  alpha=0.01
  print(isChi(text,fo,fe),"ischi")
  print(isGOF(text,fo,fe),"GOF")
  print(isTFI(text,fo,fe),"TFI")
  main(text,fo,fe)


True ischi
False GOF
True TFI
fo [[7, 18, 6, 9], [38, 23, 34, 97], [58, 47, 31, 30]]
fe [[10.35175879396985, 8.844221105527637, 7.135678391959799, 13.668341708542714], [49.688442211055275, 42.45226130653266, 34.25125628140704, 65.60804020100502], [42.959798994974875, 36.7035175879397, 29.613065326633166, 56.72361809045226]]
n 20
df 6
Starting Test for Independence (TFI) analysis...

Step 1: Chi-Square Statistic Calculation
Calculated Chi-Square Statistic: 59.8328

Step 2: P-value Calculation
Calculated P-value: 4.867e-11

Step 3: Critical Value Calculation
Calculated Critical Value: 12.5916

Step 4: Significance Testing Based on Chi-Square Statistic
Using alpha level of 0.05, the chi-square statistic result is determined to be significant.

Step 5: Significance Testing Based on P-value
Using alpha level of 0.05, the p-value result is determined to be significant.

Final Result: The association between the variables is significant based on p-value and significant based on chi-square sta